In [ ]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf
import argparse
import yaml
from pprint import pprint

In [ ]:
cur_dir = '/home/jovyan/work/projects/COSME'
config_subdir = 'configs/make_data_splits_config.yaml'

In [ ]:
config_dir = f"{cur_dir}/{config_subdir}"

In [ ]:
print(f"loading yaml file...")
config = open(config_dir, 'r').read()
pprint(config_yaml_data)

In [ ]:
in_file = config['in_file']  
out_dir = config['out_dir']  
splits = config['splits']  
rand_seed = config['rand_seed']  
do_rand_seed = config['do_rand_seed']  
CUDA_VISIBLE_DEVICES = config['CUDA_VISIBLE_DEVICES']  
do_cuda_vis_dev = config['do_cuda_vis_dev']  
partition_size = config['partition_size']  

In [ ]:
split_names = []
split_perc = []
for key, val in splits.items():
    split_names.append(key)
    split_perc.append(val)

In [ ]:
# turn off random seed if needed
if not do_rand_seed:
    rand_seed = None

In [ ]:
print(f"starting Dask GPU cluster...")
if do_cuda_vis_dev:
    cluster = LocalCUDACluster(
        protocol="ucx",
        enable_tcp_over_ucx=True,
        CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
        local_directory='/tmp',
    )
else:
    cluster = LocalCUDACluster(
        protocol="ucx",
        enable_tcp_over_ucx=True,
        local_directory='/tmp',
    )
client = Client(cluster)

In [ ]:
# first we create the Dask dataframe
print(f"reading file {in_file}")
df = dask_cudf.read_parquet(in_file,  # location of raw file
                        partition_size=partition_size,
                        )

In [ ]:
print(f"splitting data")
df_split = df.random_split(split_perc, random_state=rand_seed)

In [ ]:
for i, cur_split inenumerate(df_split):
    cur_out_file = f"{out_dir}/{split_names[i]}.parquet"
    print(f"saving {split_names[i]} data to {cur_out_file}")
    # the final step is to save the cleaned data.
    _ = df.to_parquet(cur_out_file)

In [ ]:
print(f"shutting down Dask client")
client.shutdown()
print(f"finished")